In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import re

In [2]:
import os
import sys
sys.path.insert(0, "TruthfulQA")

import torch
import torch.nn as nn
import torch.nn.functional as F
# import llama
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
import pandas as pd
import warnings
from einops import rearrange
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizer
from baukit import Trace, TraceDict
import sklearn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
import pickle
from functools import partial
from pprint import pprint
from sentence_transformers import SentenceTransformer

from truthfulqa import utilities, models, metrics
import openai
from truthfulqa.configs import BEST_COL, ANSWER_COL, INCORRECT_COL


In [3]:
import sys
sys.path.append('../')
from utils_toxic import alt_tqa_evaluate, flattened_idx_to_layer_head, layer_head_to_flattened_idx, get_interventions_dict, get_top_heads, get_separated_activations, get_com_directions
from utils_toxic import get_special_directions, get_matrix_directions

In [1]:
with open(f'./features/llama_7B_toxigen_vicuna_texts.json', 'r') as json_file:
    data = json.load(json_file)

# Save to CSV
with open('./TruthfulQA/llama_7B_toxigen_vicuna.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=["text", "toxic_text", "non_toxic_text"])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

NameError: name 'json' is not defined

In [4]:
model_name = 'llama_7B'
dataset_name = 'toxigen'
num_heads = 32

In [35]:
with open(f"/work/hdd/bcxt/yian3/toxic/features/{model_name}_{dataset_name}_categories.pkl", "rb") as f:
    categories = pickle.load(f)  # List of target groups, 1 per sentence
    
head_wise_activations = np.load(f"/work/hdd/bcxt/yian3/toxic/features/{model_name}_{dataset_name}_head_wise.npy")[:100]
head_wise_activations = rearrange(head_wise_activations, 'b l (h d) -> b l h d', h = num_heads)
labels = np.load(f"/work/hdd/bcxt/yian3/toxic/features/{model_name}_{dataset_name}_labels.npy")[:100]


In [6]:
result = pd.read_csv(f"results_dump/answer_dump/{dataset_name}_{model_name}_seed_2_top_72_heads_alpha_15_fold_0_special.csv")


In [23]:
def extract_after_second_newline(text):
    if not isinstance(text, str):
        return text  # skip if not a string
    match = re.search(r'^(?:.*?\n){2}(.*)', text, re.DOTALL)
    return match.group(1).strip() if match else text

# Apply to the 'llama_7B' column
result['llama_7B_clean'] = result['llama_7B'].apply(extract_after_second_newline)

In [ ]:
for i in range(500, 506):
    print(result['label'][i], result['text'][i])
    print("------------------------------------------")
    print(result['llama_7B_clean'][i])
    print("==========================================")

In [ ]:
result['label'][2]

In [ ]:
result['text'][2]